In [ ]:
from ultralytics import YOLO
import blobconverter

# 1. Charger le modèle pré-entraîné (transfer learning)
model = YOLO("yolo11n.pt") 

# 2. Lancer l'entraînement
# imgsz=416 est important si vous comptez déployer en 416x416 sur l'OAK-D
results = model.train(
    data="path/to/my_dataset/data.yaml",  # Chemin vers votre data.yaml
    epochs=50,                            # Nombre d'époques (ajustez selon besoin)
    imgsz=416,                            # Taille d'image cible
    batch=16,                             # Taille du batch (réduire si manque de VRAM)
    device=0,                             # 0 pour GPU, 'cpu' pour processeur
    name="yolo11n_custom"                 # Nom du dossier de sauvegarde
)

# Une fois fini, le meilleur modèle est sauvegardé ici :
# runs/detect/yolo11n_custom/weights/best.pt

In [ ]:
# 1. Charger VOS poids entraînés
my_model_path = "runs/detect/yolo11n_custom/weights/best.pt"
model = YOLO(my_model_path)

# 2. Exporter en ONNX avec OPSET 11 (Crucial pour OAK-D)
# On force imgsz=416 pour correspondre à l'entraînement
model.export(format="onnx", opset=11, imgsz=416)

# Le fichier sera généré à côté du .pt, ex: "runs/detect/yolo11n_custom/weights/best.onnx"
onnx_path = my_model_path.replace(".pt", ".onnx")

print(f"Export ONNX terminé : {onnx_path}")

# 3. Convertir en BLOB pour OAK-D
blob_path = blobconverter.from_onnx(
    model=onnx_path,
    data_type="FP16",
    shaves=6,
    use_cache=False,
    version="2022.1",
    optimizer_params=["--scale=255", "--reverse_input_channels"]
)

print(f"DONE ! Fichier Blob prêt pour la caméra : {blob_path}")
# Renommez ce fichier (ex: "custom_model.blob") et copiez-le vers votre script OAK-D

In [1]:
import blobconverter

blob_path = blobconverter.from_openvino(
    xml="yolo11n_openvino_model/yolo11n.xml",
    bin="yolo11n_openvino_model/yolo11n.bin",
    data_type="FP16",
    shaves=6,  # Utilise 6 coeurs du VPU
    use_cache=False,
    version="2022.1" # Version OpenVINO compatible
)

print(f"Modèle converti sauvegardé ici : {blob_path}")
# Renommez le fichier obtenu en "yolo11n.blob" pour simplifier

{
    "exit_code": 1,
    "message": "Command failed with exit code 1, command: /opt/intel/openvino2022_1/tools/compile_tool/compile_tool -m /tmp/blobconverter/62067fb0add44126a1a0bb2a721365a1/yolo11n/FP16/yolo11n.xml -o /tmp/blobconverter/62067fb0add44126a1a0bb2a721365a1/yolo11n/FP16/yolo11n.blob -c /tmp/blobconverter/62067fb0add44126a1a0bb2a721365a1/myriad_compile_config.txt -d MYRIAD -ip U8",
    "stderr": "Cannot create MaxPool layer __module.model.9.m/aten::max_pool2d/MaxPool id:251 from unsupported opset: opset14\n",
    "stdout": "OpenVINO Runtime version ......... 2022.1.0\nBuild ........... 2022.1.0-7019-cdb9bec7210-releases/2022/1\n"
}


HTTPError: 400 Client Error: BAD REQUEST for url: https://blobconverter.luxonis.com/compile?version=2022.1&no_cache=True

In [2]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

# On force l'opset à 11 pour la compatibilité Myriad X
# On garde imgsz=416 pour la fluidité (ou 640 si vous voulez plus précis mais plus lent)
model.export(format="onnx", imgsz=416, opset=11)

Ultralytics 8.3.225 🚀 Python-3.13.5 torch-2.8.0+cu128 CPU (Intel Core i9-14900HX)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 84, 3549) (5.4 MB)

ONNX: starting export with onnx 1.20.0 opset 11...
ONNX: slimming with onnxslim 0.1.80...
ONNX: export success ✅ 0.7s, saved as 'yolo11n.onnx' (10.2 MB)

Export complete (0.8s)
Results saved to /home/nathan/Documents/GitHub/3d_edge_object_detection
Predict:         yolo predict task=detect model=yolo11n.onnx imgsz=416  
Validate:        yolo val task=detect model=yolo11n.onnx imgsz=416 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'yolo11n.onnx'

In [ ]:
import blobconverter

blob_path = blobconverter.from_onnx(
    model="yolo11n.onnx",   # Le fichier généré à l'étape A
    data_type="FP16",       # Obligatoire pour OAK-D (Myriad X)
    shaves=6,               # 6 coeurs
    use_cache=False,
    version="2022.1",       # Version stable pour OAK-D
    optimizer_params=["--scale=255", "--reverse_input_channels"] 
    # Note: Ultralytics normalise souvent déjà, mais parfois MyriadX préfère ça.
    # Si la détection est mauvaise, essayez sans ces optimizer_params.
)

print(f"Succès ! Blob sauvegardé : {blob_path}")

[==================================================]
Done
Succès ! Blob sauvegardé : /home/nathan/.cache/blobconverter/yolo11n_openvino_2022.1_6shave.blob
